En la lección de hoy aprendimos como crearnos una clase que nos permita limpiar los datos obtenidos de la API.
En este ejercicio, tendréis que crear una clase con el código que usamos en los ejercicios de pair programming de ETL Transformación I y II.

In [ ]:
from IPython.core.interactiveshell import InteractiveShell 
InteractiveShell.ast_node_interactivity = "all"

import requests
import pandas as pd
import numpy as np
from datetime import datetime

In [ ]:
#pd.options.display.float_format = '{:.2f}'.format

In [ ]:
class Energias:

    def __init__(self, inicio, fin):
        self.inicio =  inicio
        self.fin = fin

    def extraer_tipos(self):
        datos_energia = pd.DataFrame()
        for i in range(self.inicio, self.fin):
            url = f"https://apidatos.ree.es/es/datos/generacion/evolucion-renovable-no-renovable?start_date={i}-01-01T00:00&end_date={i}-12-31T23:59&time_trunc=year"
            response = requests.get(url=url)
            datos = response.json()

            for e in range(len(datos["included"])):
                x = pd.DataFrame(datos["included"][e]["attributes"]["values"])
                x["tipo_energia"] = datos["included"][e]["attributes"]["title"]

                datos_energia = pd.concat([datos_energia, x], axis=0, ignore_index = True)

        return datos_energia
    
    def energia_comunidades(self, claves):
        self.claves = claves
        datos_energia = pd.DataFrame()
        for i in range(self.inicio, self.fin):
            for k,v in claves.items():
                url = f"https://apidatos.ree.es/es/datos/generacion/evolucion-renovable-no-renovable?start_date={i}-01-01T00:00&end_date={i}-12-31T23:59&time_trunc=year&geo_trunc=electric_system&geo_limit=ccaa&geo_ids={v}"
                response = requests.get(url=url)
                datos = response.json()

                for e in range(len(datos["included"])):
                    x = pd.DataFrame(datos["included"][e]["attributes"]["values"])
                    x["tipo_energia"] = datos["included"][e]["attributes"]["title"]
                    x["comunidad"] = k
                    x["cod_comunidad"] = v

                    datos_energia = pd.concat([datos_energia, x], axis=0, ignore_index = True)

        return datos_energia
    
    def redondear_valores(self, df):
        self.df =df
        df["value"] = round(df["value"], 2)
        df["percentage"] = round(df["percentage"], 2)
        return df 
    
    def sacar_fecha(self, df):
        self.df =df
        df["fecha"] = df["datetime"]
        df["fecha"] = df["fecha"].apply(lambda tiempo: tiempo.split("T")[0])
        df["fecha"] = df["fecha"].apply(pd.to_datetime)
        #print(type(df["fecha"]))
        #df["fecha"] = df["fecha"].apply(datetime.strftime("%Y-%m-%d"))
        #return df 
    
    def guardar_datos(self, df, ruta, nombre):
        self.ruta = ruta
        self.df =df
        self.nombre = nombre

        df.to_pickle(ruta + nombre + ".pkl")
        df.to_csv(ruta + nombre + ".csv")

In [ ]:
cod_comunidades = {'Ceuta': 8744,
                    'Melilla': 8745,
                    'Andalucía': 4,
                    'Aragón': 5,
                    'Cantabria': 6,
                    'Castilla - La Mancha': 7,
                    'Castilla y León': 8,
                    'Cataluña': 9,
                    'País Vasco': 10,
                    'Principado de Asturias': 11,
                    'Comunidad de Madrid': 13,
                    'Comunidad Foral de Navarra': 14,
                    'Comunitat Valenciana': 15,
                    'Extremadura': 16,
                    'Galicia': 17,
                    'Illes Balears': 8743,
                    'Canarias': 8742,
                    'Región de Murcia': 21,
                    'La Rioja': 20}

In [ ]:
energy = Energias(2011, 2023)

In [ ]:
energias_renovables = energy.extraer_tipos()

In [ ]:
energias_renovables.head()

In [ ]:
tipos_por_comunidades = energy.energia_comunidades(cod_comunidades)

In [ ]:
tipos_por_comunidades.head()

In [ ]:
energias_renovables_round = energy.redondear_valores(energias_renovables)

In [ ]:
energias_renovables_round.head(1)

In [ ]:
tipos_por_comunidades_round = energy.redondear_valores(tipos_por_comunidades)

In [ ]:
tipos_por_comunidades_round.head()

In [ ]:
energias_renovables_round["fecha"].apply(lambda x: print(type(x)))

In [ ]:
energias_renovables_fecha = energy.sacar_fecha(energias_renovables_round)

In [ ]:
energias_renovables_fecha.head()

In [ ]:
tipos_por_comunidades_fecha = energy.sacar_fecha(tipos_por_comunidades_round)

In [ ]:
guardado_tipos = energy.guardar_datos(energias_renovables_fecha, "datos/", "tipo_energia")

In [ ]:
guardado_comunidades = energy.guardar_datos(tipos_por_comunidades_fecha, "datos/", "datos_comunidades")